In [ ]:

# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('ggplot')


In [ ]:

# Cargar los datos originales
file_2015 = "yellow_tripdata_201501_reducido.csv"
file_2016 = "yellow_tripdata_201601_reducido.csv"

df_2015 = pd.read_csv(file_2015)
df_2016 = pd.read_csv(file_2016)

# Concatenar ambos datasets
df = pd.concat([df_2015, df_2016], ignore_index=True)

# Mostrar estructura del dataset
df.head()


In [ ]:

# Análisis de valores nulos y duplicados
print("Valores Nulos por columna:")
print(df.isnull().sum())

print("\nNúmero de filas duplicadas:", df.duplicated().sum())


In [ ]:

# Estadísticas descriptivas
df.describe()


In [ ]:

# Visualización de la distribución de variables numéricas
df.hist(bins=30, figsize=(15, 10), layout=(6, 4))
plt.suptitle("Distribución de Variables Numéricas", fontsize=16)
plt.show()


In [ ]:

# Boxplots para detección de outliers
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

plt.figure(figsize=(15, 10))
for col in numeric_cols:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df[col])
    plt.title(f"Boxplot de {col}")
    plt.show()


In [ ]:

# Matriz de Correlación
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación")
plt.show()


In [ ]:

# Limpieza y Transformación de Datos

# Convertir fechas a formato datetime
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"], errors='coerce')
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"], errors='coerce')

# Crear nuevas columnas
df["trip_duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
df["speed_mph"] = (df["trip_distance"] / df["trip_duration"]) * 60

# Manejo de valores extremos en velocidad
df["speed_mph"] = df["speed_mph"].replace([np.inf, -np.inf], np.nan)
df["speed_mph"] = df["speed_mph"].fillna(df["speed_mph"].median())

# Variables adicionales
df["is_weekend"] = df["tpep_pickup_datetime"].dt.weekday.isin([5, 6]).astype(int)
df["rush_hour"] = df["tpep_pickup_datetime"].dt.hour.isin([7, 8, 9, 16, 17, 18]).astype(int)
df["fare_per_mile"] = df["fare_amount"] / df["trip_distance"]

# Manejo de valores nulos en `fare_per_mile`
df["fare_per_mile"] = df["fare_per_mile"].replace([np.inf, -np.inf], np.nan)
df["fare_per_mile"] = df["fare_per_mile"].fillna(df["fare_per_mile"].median())

# Extraer año y fecha del viaje
df["Year"] = df["tpep_pickup_datetime"].dt.year
df["pickup_date"] = df["tpep_pickup_datetime"].dt.date

# Redondear valores numéricos
numeric_cols = df.select_dtypes(include=['float', 'int']).columns
df[numeric_cols] = df[numeric_cols].round(2)

# Mostrar las nuevas columnas creadas
df.head()


In [ ]:

# Guardar el dataset limpio
df.to_csv("cleaned_taxi_data.csv", index=False)
print("Dataset limpio guardado como 'cleaned_taxi_data.csv'")
